In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

mnist = input_data.read_data_sets("/path/to/MNIST_data",one_hot = True)



Extracting /path/to/MNIST_data\train-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data\train-labels-idx1-ubyte.gz
Extracting /path/to/MNIST_data\t10k-images-idx3-ubyte.gz
Extracting /path/to/MNIST_data\t10k-labels-idx1-ubyte.gz
Training data size: 55000


In [7]:
print("Training data size:",mnist.train.num_examples)
print("Validating data size",mnist.validation.num_examples)
print("Test data size:",mnist.test.num_examples)

print("Training data:",mnist.train.images[0])
print("Training data label:",mnist.train.labels[0])





Training data size: 55000
Validating data size 5000
Test data size: 10000
Training data: [0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0. 

In [9]:
batch_size = 100
xs,ys = mnist.train.next_batch(batch_size)
print("x shape:",xs.shape)
print("Y shape:",ys.shape)

x shape: (100, 784)
Y shape: (100, 10)


In [3]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

# 输入的节点数量，等于图片的像素
INPUT_NODE = 784
# 输出的节点数（0-9）10个数字
OUTPUT_NODE = 10

# 隐藏层的网络结构，500个节点
LAYER1_NODE = 500
# 一个batch 中的训练数据的个数，
BATCH_SIZE = 100

# 基础的学习率
LEARNING_RATE_BASE = 0.8
# 学习率的衰减
LEARNING_RATE_DECAY = 0.99
# 正则化项的损失函数中的系数
REGULARIZATION_RATE = 0.0001
# 训练步数
TRAINING_STEPS = 30000
# 滑动平均衰减率
MOVING_AVERAGE_DECAY = 0.99

# 计算前向传播的结果
def inference0(input_tensor,avg_class,weights1,biases1,weights2,biases2):
    if avg_class == None:
        # relu激活函数
        layer1 = tf.nn.relu(tf.matmul(input_tensor,weights1)+biases1)
        # 计算输出前向传播的结果，这里没有加入softmax函数
        # 因为损失函数会一并计算softmaxe
        return tf.matmul(layer1,weights2) + biases2
    else: # 计算滑动平均参数
        layer1 = tf.nn.relu(tf.matmul(input_tensor,avg_class.average(weights1))+avg_class.average(biases1))
        return tf.matmul(layer1,avg_class.average(weights2)) + avg_class.average(biases2)

    
def inference(input_tensor,reuse = False):
    with tf.variable_scope('layer1',reuse = reuse):
        weights = tf.get_variable("weights",[INPUT_NODE,LAYER1_NODE],
                                  initializer=tf.truncated_normal_initializer(stddev = 0.1))
        biases = tf.get_variable("biases",[LAYER1_NODE],
                                initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor,weights)+biases)
        
    with tf.variable_scope('layer2',reuse = reuse):
        weights = tf.get_variable("weights",[LAYER1_NODE,OUTPUT_NODE],
                                  initializer=tf.truncated_normal_initializer(stddev = 0.1))
        biases = tf.get_variable("biases",[OUTPUT_NODE],
                                initializer=tf.constant_initializer(0.0))
        layer1 = tf.matmul(layer1,weights)+biases
    
# 训练模型
def train(mnist):
    
    # 输入数据
    x  = tf.placeholder(tf.float32,[None,INPUT_NODE],name='x-input')
    y_ = tf.placeholder(tf.float32,[None,OUTPUT_NODE],name = 'y-input')
    
    # 生成隐藏参数
    weights1 = tf.Variable(tf.truncated_normal([INPUT_NODE,LAYER1_NODE],stddev = 0.1))
    biase1 = tf.Variable(tf.constant(0.1,shape=[LAYER1_NODE]))
    # 生成输出参数
    weights2 = tf.Variable(tf.truncated_normal([LAYER1_NODE,OUTPUT_NODE],stddev = 0.1))
    biases2 = tf.Variable(tf.constant(0.1,shape =[OUTPUT_NODE]))
    
    # 计算前向传播的结果
    # y = inference(x,None,weights1,biase1,weights2,biases2)
    y = inference(x,reuse = True)
    
    # 定义存储轮数的变量，指定不可训练的变量
    global_step = tf.Variable(0,trainable=False)
    
    # 给定滑动平均衰减率和训练轮数的变量
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY,global_step)
    # 
    variable_averages_op = variable_averages.apply(tf.trainable_variables())
    # 滑动平均之后的前向传播结果
    average_y = inference(x,variable_averages,weights1,biase1,weights2,biases2)
    
    # 计算交叉熵
    corss_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = y,labels=tf.argmax(y_,1))
    # 计算当前batch中所有样例的交叉熵的平均值
    cross_entropy_mean = tf.reduce_mean(corss_entropy)
    
    # 计算L2 正则化损失函数
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    # 计算模型的正则化损失
    regularization = regularizer(weights1) + regularizer(weights2)
    # 总损失需要加上正则化的损失
    loss = cross_entropy_mean + regularization
    
    # 设置衰减的学习率
    learning_rate = tf.train.exponential_decay(
                LEARNING_RATE_BASE, #基础学习率
                global_step, # 当前迭代的轮数
                mnist.train.num_examples / BATCH_SIZE, #需要迭代的次数
                LEARNING_RATE_DECAY # 学习率衰减的速度
                )
    
    
    # 优化算法来优化损失函数
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss,global_step=global_step)

    with tf.control_dependencies([train_step,variable_averages_op]):
        train_op = tf.no_op(name = 'train')

    correct_prediction = tf.equal(tf.arg_max(average_y,1),tf.argmax(y_,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction,tf.float32))


    with tf.Session() as sess:
        tf.global_variables_initializer().run()

        validate_feed = {x:mnist.validation.images,
                        y_:mnist.validation.labels}

        test_feed = {x:mnist.test.images,y_:mnist.test.labels}

        for i in range(TRAINING_STEPS):
            if i % 1000 == 0:
                validate_acc = sess.run(accuracy,feed_dict=validate_feed)
                print("%d steps, validation acc" "using average model is %g" %(i,validate_acc))

            xs,ys = mnist.train.next_batch(BATCH_SIZE)
            sess.run(train_op,feed_dict={x:xs,y_:ys})

        test_acc = sess.run(accuracy,feed_dict = test_feed)
        print("%d steps, test acc" "model is %g" %(TRAINING_STEPS,test_acc))
    
    

def main(argv = None):
    mnist = input_data.read_data_sets("/tmp/data",one_hot = True)
    train(mnist)
    
if __name__ == '__main__':
    tf.app.run()

        

        






Extracting /tmp/data\train-images-idx3-ubyte.gz
Extracting /tmp/data\train-labels-idx1-ubyte.gz
Extracting /tmp/data\t10k-images-idx3-ubyte.gz
Extracting /tmp/data\t10k-labels-idx1-ubyte.gz
Instructions for updating:
Use `tf.math.argmax` instead
0 steps, validation accusing average model is 0.1046
1000 steps, validation accusing average model is 0.9762
2000 steps, validation accusing average model is 0.981
3000 steps, validation accusing average model is 0.9834
4000 steps, validation accusing average model is 0.9836
5000 steps, validation accusing average model is 0.9854
6000 steps, validation accusing average model is 0.9856
7000 steps, validation accusing average model is 0.9856
8000 steps, validation accusing average model is 0.9838
9000 steps, validation accusing average model is 0.985
10000 steps, validation accusing average model is 0.986
11000 steps, validation accusing average model is 0.9858
12000 steps, validation accusing average model is 0.9854
13000 steps, validation accus

SystemExit: 

D:\Anaconda3\envs\tfgpu\lib\site-packages\IPython\core\interactiveshell.py:3304: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


##  变量的管理

 TensorFlow 提供了通过变量名称来创建或者获取一个变量 的机制 。 通过这个机制，在不同的函数中可．以直接通过变量 的名字来使用变量 ，而不需要将变盘通过参数的形式到处传递 。 

TensorFlow 中通过变 量 名称获取变量的机制主要是通过 
- `tf.get_variable`
    - 函数来创建或者获取变量。
    - 必须指定变量名称的参数，会根据这个名字去创建或者获取变量 。 
- `tf.variable_scope`



![initializer.png](imgs/initializer.png)

 如果需要通过tf.get_variable 获取一个已经创建的变量，需要通过 tf.variable_scope 函数来生成一个上下文管理器，并明确指定在这个上下文管理器中，tf.get_variable将直接获取己经生成的变量。
 
 tf.variable_scope:变量作用域定义。 函数可以控制 tf.get_variable 函数的语义，函数使用参数 reuse=True 生成上下文管理器时，这个上下文管理器内所有的 tf.get_variable函数会直接获取己经创建的变量。
 
 如果 tf.variable_scope 函数使用参数 reuse=None 或者 reuse=False 创建上下文管理器tf.get_variable 操作将创建新的变量。
 
 


In [3]:
# tf.getvariable
import tensorflow as tf
v1 = tf.get_variable("v1",shape=[1],initializer=tf.constant_initializer(1.0))
v2 = tf.Variable(tf.constant(1.0,shape=[1]),name = "v2")
print(v2)
print(v1)
    

<tf.Variable 'v2_1:0' shape=(1,) dtype=float32_ref>
<tf.Variable 'v1:0' shape=(1,) dtype=float32_ref>


In [11]:


import tensorflow as tf
tf.reset_default_graph() #每次运行将图中的变量清空
with tf.variable_scope("foo"):
    v1 = tf.get_variable("v",[2],initializer=tf.constant_initializer(1.0))
    print("v:", v.name)
with tf.variable_scope("foo1"):
     v2 = tf.get_variable("v",[2],initializer=tf.constant_initializer(3.0))
     print("v:", v.name)
    
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    print("v1=",v1.eval())
    print("v2=",v2.eval())

    

v: foo1/v:0
v: foo1/v:0
v1= [1. 1.]
v2= [3. 3.]


In [2]:
#在生成上下文管理器，将参数reuse 设置为True,就可以直接获得已经声明的参数
with tf.variable_scope("foo",reuse = True):
    v1 = tf.get_variable("v",[1])
    print (v == v1)

True


## TensorFlow 模型的保存和调用

### 模型的保存和调用的实现

**模型的保存的方法**
- 创建Saver:  `saver = tf.train.Saver()`
- 保存Saver: `saver.save(sess,"F:/Study/Tfcook/tensorflowcook/03_MNIST/model/model.ckpt")`
    - 要在Session中执行save
- 保存的文件：
    -  model.ckpt.meta ： 保存了 TensorFlow 计算图的结构 
    - model.ckpt: 保存了 TensorFlow 程序中每一个变量 的取值。
    - chekpoint: 保存了一个目录下所有的模型文件列表
    
![saveModel.png](imgs/saveModel.png)

**模型的加载的方法1**
- 定义了 TensorFlow 计算图上的所有运算 
- 创建Saver:  `saver = tf.train.Saver()`
- 加载Saver:`saver.restore(sess,"F:/Study/Tfcook/tensorflowcook/03_MNIST/model/model.ckpt")`


**模型的加载的方法2**
- 不希望重复定义图上的运算
- saver = tf.train.import_meta_graph(
    "F:/Study/Tfcook/tensorflowcook/03_MNIST/model/model.ckpt.meta")
- saver.restore(sess,"F:/Study/Tfcook/tensorflowcook/03_MNIST/model/model.ckpt")
- sess.run(tf.get_default_graph().get_tensor_by_name("add:0"))

只要保存神经网络的输入层经过前向传播计算得到输出层
-  `convert_variables_to_constants`函数：将计算图中的变量及其取值通过常量的方式保存
-  计算图可以统一存放在一个文件中


In [10]:
# 模型的保存
import tensorflow as tf

tf.reset_default_graph()

v1 = tf.Variable(tf.constant(1.0,shape = [1]),name = "v1")
v2 = tf.Variable(tf.constant(2.0,shape = [1]),name = "v2")

result = v1 + v2

init_op = tf.global_variables_initializer()
# tf.train.Saver 用于保存模型
saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(init_op)
    
    saver.save(sess,"F:/Study/Tfcook/tensorflowcook/03_MNIST/model/model.ckpt")


In [13]:
# 加载模型实例
# 定义了TnsorFlow计算图上的所有运算

import tensorflow as tf

tf.reset_default_graph() #将图中的变量清空

v1 = tf.Variable(tf.constant(1.0,shape = [1]),name = "v1")
v2 = tf.Variable(tf.constant(2.0,shape = [1]),name = "v2")

result = v1 + v2


# tf.train.Saver 用于加载模型
saver = tf.train.Saver()

with tf.Session() as sess:
    saver.restore(sess,"F:/Study/Tfcook/tensorflowcook/03_MNIST/model/model.ckpt")
    print(sess.run(result))





INFO:tensorflow:Restoring parameters from F:/Study/Tfcook/tensorflowcook/03_MNIST/model/model.ckpt
[3.]


In [26]:
# 直接加载模型
import tensorflow as tf

tf.reset_default_graph() #将图中的变量清空

saver = tf.train.import_meta_graph(
    "F:/Study/Tfcook/tensorflowcook/03_MNIST/model/model.ckpt.meta")
with tf.Session() as sess:
    saver.restore(sess,"F:/Study/Tfcook/tensorflowcook/03_MNIST/model/model.ckpt")
    print (sess.run(tf.get_default_graph().get_tensor_by_name("add:0")))


INFO:tensorflow:Restoring parameters from F:/Study/Tfcook/tensorflowcook/03_MNIST/model/model.ckpt
[3.]


In [28]:
# convert_variables_to_constants

import tensorflow as tf
from tensorflow.python.framework import graph_util

tf.reset_default_graph() #将图中的变量清空

v1 = tf.Variable(tf.constant(1.0,shape = [1]),name = "v1")
v2 = tf.Variable(tf.constant(2.0,shape = [1]),name = "v2")
result = v1 + v2

init_op = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init_op)
    #导出当前计算图的 GraphDef 部分，只需要这一部分就可以完成从输入层到输出层的计算过程。
    graph_def = tf.get_default_graph().as_graph_def()
    #＃将圈中的变量及其取值转化为常量，同时将图中不必要的节点去掉。
    #将会看到一些系统运算也会被转化为计算图中的节点（比如变量初始化操作）。
    # 在下面一行代码中，后一个参数［＇ add＇］给出了需要保存的节点名称。
    # 注意这里给出的是计算节点的名称,所以没有后面的： 0
    output_graph_def = graph_util.convert_variables_to_constants(
    sess,graph_def,['add'])
    
    with tf.gfile.GFile("F:/Study/Tfcook/tensorflowcook/03_MNIST/model/combined_model.pb","wb") as f:
        f.write(output_graph_def.SerializeToString())



Instructions for updating:
Use tf.compat.v1.graph_util.convert_variables_to_constants
Instructions for updating:
Use tf.compat.v1.graph_util.extract_sub_graph
INFO:tensorflow:Froze 2 variables.
INFO:tensorflow:Converted 2 variables to const ops.


In [30]:
import tensorflow as tf
from tensorflow.python.platform import gfile
tf.reset_default_graph() #将图中的变量清空


with tf.Session() as sess:
    model_filename = "F:/Study/Tfcook/tensorflowcook/03_MNIST/model/combined_model.pb"
    with gfile.FastGFile(model_filename,'rb') as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
        
    result = tf.import_graph_def(graph_def,return_elements =["add:0"])
    
    print(sess.run(result))

[array([3.], dtype=float32)]


### 模型保存原理及数据格式

TensorFlow 通过元图(MetaGraph ）来记录计算图中节点的信息以及运行计算图中节点所需要的元数据。
元图是由 MetaGraphDef Protocol Buffer 定义的。


 #### MetaGraphDef 文件
 ![metaGraph.png](imgs/metaGraph.png)
 
 元图中主要记录了6类信息,对应的文件就是：model.ckpt.meta -- 是一个二进制文件，无法直接查看。
 可以通过`export_meta _graph`查看
 
 **meta_info_def属性**
 
 通过 MetalnfoDef 定义的，它记录了 TensorFlow 计算图中的元数据以及 TensorFlow 程序中所有使用到的运算方法的信息 .
 ![metaInfoDef.png](imgs/metaInfoDef.png)
 
 了计算图的版本号（ meta_graph_ version 属性）以及用户指定的一些标签（ tags 属性）。如果没有在 saver 中特殊指定，那么这些属性都默认为 空。 meta_info_def 属性里只有 stripped_op_list 属性是不为空的。
 - 记录了 TensorFlow 计算图上使用到的所有运算方法的信息 。
 -  stripped_op_list 也只会出现一次
 - 由stripped_op_list 属性的类型是 OpList 
     - 第一个属性 name定义了运算的名称
     - 第二和第三个属性为 input_arg 和 output_arg，它们定义了运算的输入和输出 
     - 第 四个属性attr给出了其他的运算参数信息
     
```
op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_BFLOAT16
            type: DT_HALF
            type: DT_FLOAT

```

名称为 Add 的运算。这个运算有 2 个输入和 1 个输出，输入输出属性都指定了属性 type_attr ，并且这个属性的值为 T 。在 OpDef 的 attr 属性中，必须要出现名称(name ）为 T 的属性。

MetalnfoDef 中的 tensorflow_version 和 tensorflow_git_version 属性记录了生成当前计算图的 TensorFlow 版本。
     
**graph_def属性**

主要记录了 TensorFlow 计算图上的节点信息。只关注运算的连接结构。要包含了 一个 NodeDef 类型的列表
![graphDef.png](imgs/graphDef.png)
![graphDef2.png](imgs/graphDef2.png)

- GraphDef 中的 versions 属性比较简单，它主要存储了 TensorFlow 的版本号 
- NodeDef 类型中有一 个名称属性 name，它是一 个节点的唯一标识符
    - NodeDef 类型中的 input 属性是一个字符串列表，它定义了运算的输入。
    - device 属性指定了处理这个运算的设备。
    - NodeDef 类型中的 attr 属性指定了和当前运算相关的配置信息。
```
 node {
    name: "v1"
    op: "VariableV2"
    attr {
      key: "_output_shapes"
      value {
        list {
          shape {
            dim {
              size: 1
            }
          }
        }
      }
    }
    attr {
      key: "container"
      value {
        s: ""
      }
    }
    attr {
      key: "dtype"
      value {
        type: DT_FLOAT
      }
    }
    attr {
      key: "shape"
      value {
        shape {
          dim {
            size: 1
          }
        }
      }
    }
    attr {
      key: "shared_name"
      value {
        s: ""
      }
    }
  }

```
 
 **saver_def属性**
 
 saver_def 属性中记录了持久化模型时需要用到的一些参数，比如保存到文件的文件名、保存操作和加载操作的名称以及保存频率、清理历史记录等。
 ![saverDef01.png](imgs/saverDef01.png)
 ![saverDef01.png](imgs/saverDef01.png)
 
 ```
 saver_def {
  filename_tensor_name: "save/Const:0"
  save_tensor_name: "save/control_dependency:0"
  restore_op_name: "save/restore_all"
  max_to_keep: 5
  keep_checkpoint_every_n_hours: 10000.0
  version: V2
}
 ```
 
 - filename_tensor_name 属性给出了保存文件名的张量名称
 - save_tensor_name 属性给出了持久化 TensorFlow 模型的运算所对应的节点名称
 - restore_op_name  加载 TensorFlow 模型的运算
 -  max_to_ keep 属性和keep_checkpoint_every_n_hours 属性设定了 tf.train.Saver 类清理之前保存的模型的策略
 
 **collection_def属性**
 
 是一个从集合名称到集合内容的映射，其中集合名称为字符串 
 ![collectionDef.png](imgs/collectionDef.png)
 
 计算图上的集合主要可以维护 4 类不同的集合
 - NodeList 用于维护计算图上节点的集合。
 - BytesList 可以维护字符串或者系列化之后的Procotol  Buffer 的集合 。 张量的集合是通过BytesList维护的
 - Int64List 用于维护整 数 集 合，
 - F loatList 用于维护实数集合 
 
 ```
 collection_def {
  key: "trainable_variables"
  value {
    bytes_list {
      value: "\n\004v1:0\022\tv1/Assign\032\tv1/read:02\007Const:08\001"
      value: "\n\004v2:0\022\tv2/Assign\032\tv2/read:02\tConst_1:08\001"
    }
  }
}
collection_def {
  key: "variables"
  value {
    bytes_list {
      value: "\n\004v1:0\022\tv1/Assign\032\tv1/read:02\007Const:08\001"
      value: "\n\004v2:0\022\tv2/Assign\032\tv2/read:02\tConst_1:08\001"
    }
  }
}
 
 ```
 
  一个是所有变量的集合，这个集合的名称为 variables 。另 外一个是可训练变量 的集合，名为 trainable variables 
  
  
 #### 变量文件
 
  model.ckpt.index 和model.ckpt.data-*****－of－*** 文件就保存了所有变量的取值
  model.ckpt.data 文件是通过 SSTable 格式存储的，可以大致理解为就是一个（ key, value ）列表
  - `tf.train.NewCheckpointReader` 类
  
  
  ####  checkpoint文件
  
  这个文件是 tf.train.Saver 类自动生成且自动维护的。在 checkpoint 文件中维护了由一个 tf.train.Saver 类持久化的所有 TensorFlow 模型文件的文件名。
 



In [31]:
import tensorflow as tf

tf.reset_default_graph() #将图中的变量清空

v1 = tf.Variable(tf.constant(1.0,shape = [1]),name = "v1")
v2 = tf.Variable(tf.constant(2.0,shape = [1]),name = "v2")
result = v1 + v2

saver = tf.train.Saver()

saver.export_meta_graph("F:/Study/Tfcook/tensorflowcook/03_MNIST/model/model.ckpt.meda.json",as_text=True)



meta_info_def {
  stripped_op_list {
    op {
      name: "Add"
      input_arg {
        name: "x"
        type_attr: "T"
      }
      input_arg {
        name: "y"
        type_attr: "T"
      }
      output_arg {
        name: "z"
        type_attr: "T"
      }
      attr {
        name: "T"
        type: "type"
        allowed_values {
          list {
            type: DT_BFLOAT16
            type: DT_HALF
            type: DT_FLOAT
            type: DT_DOUBLE
            type: DT_UINT8
            type: DT_INT8
            type: DT_INT16
            type: DT_INT32
            type: DT_INT64
            type: DT_COMPLEX64
            type: DT_COMPLEX128
            type: DT_STRING
          }
        }
      }
    }
    op {
      name: "Assign"
      input_arg {
        name: "ref"
        type_attr: "T"
        is_ref: true
      }
      input_arg {
        name: "value"
        type_attr: "T"
      }
      output_arg {
        name: "output_ref"
        type_attr: "T"
        

In [32]:
import tensorflow as tf

tf.reset_default_graph() #将图中的变量清空
#  tf.train.NewCheckpointReader 可以读取 checkpoint 文件中保存的所有变量。
reader = tf.train.NewCheckpointReader('F:/Study/Tfcook/tensorflowcook/03_MNIST/model/model.ckpt')

global_variables = reader.get_variable_to_shape_map()
for var_name in global_variables:
    print(var_name,global_variables[var_name])
print("Value for v1 is",reader.get_tensor("v1"))


v1 [1]
v2 [1]
Value for v1 is [1.]
